In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from timeit import default_timer as timer
from joblib import Parallel, delayed
import scipy
from sklearn import metrics
import multiprocessing
from multiprocessing import set_start_method, Pool, Process, managers
import threading, queue

In [76]:
def fac_tuple(tup_1, tup_2):
    result = np.math.factorial(tup_1) + np.math.factorial(tup_2)
    return result

In [414]:
lenght = 100
num_iteration = 10

In [400]:
list_one = [5] * lenght
list_two = [10] * lenght
list_tuples = list(zip(list_one, list_two))

# pool

In [401]:
pool = Pool(4)
time_list = []

for _ in range(num_iteration):
    start = timer()
    individuals_fitness = pool.starmap(fac_tuple, list_tuples, chunksize=1000)
    end = timer()
    time_list.append(end-start)
    
print("The average time elapsed is " + str(np.mean(time_list)))     

The average time elapsed is 0.04886552159969142


Process ForkPoolWorker-546:
Process ForkPoolWorker-547:
Process ForkPoolWorker-545:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/thomas/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/thomas/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/thomas/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-544:
  File "/home/thomas/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/thomas/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/thomas/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
 

# sequential

In [402]:
time_list = []

for _ in range(num_iteration):

    individuals_fitness = []
    start = timer()
    for index in range(lenght):
        result = fac_tuple(list_one[index], list_two[index])
        individuals_fitness.append(result)
        
    end = timer()
    time_list.append(end-start)

print("The average time elapsed is " + str(np.mean(end-start)))  

The average time elapsed is 0.0669143039995106


# multiprocess

In [403]:
num_processes = 2
num_obs_process = int(lenght/num_processes)

In [404]:
def fac_tuple_process(tup_1, tup_2, work_id, dic):
    
    list_one_split = list_one[work_id*num_obs_process:(work_id+1)*num_obs_process]
    list_two_split = list_two[work_id*num_obs_process:(work_id+1)*num_obs_process]
    
    list_result = []
    for index in range(num_obs_process):
        result = np.math.factorial(list_one_split[index]) + np.math.factorial(list_two_split[index])
        list_result.append(result)
        
    dic[work_id] = list_result

In [405]:
time_list = []

manager = multiprocessing.Manager()
return_dict = manager.dict()
    
for _ in range(10):
    start = timer()

    p = [multiprocessing.Process(target=fac_tuple_process, args=(list_one, list_two, i, return_dict)) 
         for i in range(num_processes)]
        
    for proc in p:
        proc.start()
    for proc in p:
        proc.join()

    individuals_fitness = return_dict.values()
    #individuals_fitness = [element for sub_list in individuals_fitness for element in sub_list]
    
    end = timer()
    time_list.append(end-start)
    
print("The average time elapsed is " + str(np.mean(time_list)))


The average time elapsed is 0.12269182649997674


In [406]:
for proc in p:
    proc.terminate()

# threads

In [416]:
num_processes = 4
num_obs_process = int(lenght/num_processes)

In [417]:
def fac_tuple_process(tup_1, tup_2, work_id, q):
    
    list_one_split = list_one[work_id*num_obs_process:(work_id+1)*num_obs_process]
    list_two_split = list_two[work_id*num_obs_process:(work_id+1)*num_obs_process]
    
    list_result = []
    for index in range(num_obs_process):
        result = np.math.factorial(list_one_split[index]) + np.math.factorial(list_two_split[index])
        list_result.append(result)
        
    q.put(list_result)

In [430]:
time_list = []
    
for _ in range(1):
    q = queue.Queue()
    start = timer()

    t = [threading.Thread(target=fac_tuple_process, args=(list_one, list_two, i, q)) 
         for i in range(num_processes)]
        
    for thread in t:
        thread.start()

    individuals_fitness = [q.get() for _ in range(num_processes)]
    #individuals_fitness = [element for sub_list in individuals_fitness for element in sub_list]
    
    end = timer()
    time_list.append(end-start)
    
print("The average time elapsed is " + str(np.mean(time_list)))

The average time elapsed is 0.0016812800022307783
